Animal Classification with CNN  

- The objective of this project is to explore how image manipulation techniques, particularly brightness and contrast adjusmants, impact the performanve neural network model in image classification tasks.

The model is tested on three sets:

* Original Test Set – Standard images.  
* Manipulated Test Set – Images with brightness/contrast changes.  
* Color-Corrected Test Set – Images processed with the Gray World Algorithm for color constancy.  

Key Steps:
* Data Preprocessing: Resize, normalize, and split data.  
* Model Training: Train a basic CNN model.  
* Evaluation: Compare performance across the three test sets.  

Dataset: https://www.kaggle.com/datasets/rrebirrth/animals-with-attributes-2/data


# 1. Read and Prepare Dataset

In [1]:
import os
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt

In [2]:
source = "/kaggle/input/animals-with-attributes-2/Animals_with_Attributes2/JPEGImages"  
target = "/kaggle/working/FilteredImages"

classes = ["collie", "dolphin", "elephant", "fox", "moose", "rabbit", "sheep", "squirrel", "giant+panda", "polar+bear"]
images_per_class = 650

# Create the target directory
os.makedirs(target, exist_ok=True)

# Loop through each class and copy the images
for class_name in classes:
    class_path = os.path.join(source, class_name)
    target_path = os.path.join(target, class_name)
    
    if not os.path.exists(class_path):
        print(f"Source class path {class_path} does not exist. Skipping...")
        continue
    
    # Create a subdirectory for each class in the target directory
    os.makedirs(target_path, exist_ok=True)
    
    print(f"Processing class: {class_name}")
    image_count = 0
    available_files = os.listdir(class_path)
    
    # Adjust images_per_class to the number of available images
    total_images = len(available_files)
    images_to_copy = min(images_per_class, total_images)
    print(f"Found {total_images} images. Attempting to copy {images_to_copy} images.")
    
    for file_name in available_files:
        if image_count >= images_to_copy:
            break
        
        full_file_name = os.path.join(class_path, file_name)
        if os.path.isfile(full_file_name):
            img = cv2.imread(full_file_name)
            
            if img is not None:  # Ensure the image is read properly
                cv2.imwrite(os.path.join(target_path, file_name), img)
                image_count += 1
            else:
                print(f"Warning: Unable to read image {full_file_name}")
    
    print(f"Completed {image_count}/{images_to_copy} images for class {class_name}")

Processing class: collie
Found 1028 images. Attempting to copy 650 images.
Completed 650/650 images for class collie
Processing class: dolphin
Found 946 images. Attempting to copy 650 images.
Completed 650/650 images for class dolphin
Processing class: elephant
Found 1038 images. Attempting to copy 650 images.
Completed 650/650 images for class elephant
Processing class: fox
Found 664 images. Attempting to copy 650 images.
Completed 650/650 images for class fox
Processing class: moose
Found 704 images. Attempting to copy 650 images.
Completed 650/650 images for class moose
Processing class: rabbit
Found 1088 images. Attempting to copy 650 images.
Completed 650/650 images for class rabbit
Processing class: sheep
Found 1420 images. Attempting to copy 650 images.
Completed 650/650 images for class sheep
Processing class: squirrel
Found 1200 images. Attempting to copy 650 images.
Completed 650/650 images for class squirrel
Processing class: giant+panda
Found 874 images. Attempting to copy 

In [3]:
def load_and_process_images(data_dir, image_size=(128, 128)):
    images = []
    labels = []
    for class_name in os.listdir(data_dir):
        class_path = os.path.join(data_dir, class_name)
        if os.path.isdir(class_path):  # Ensure it's a directory
            for file_name in os.listdir(class_path):
                file_path = os.path.join(class_path, file_name)
                try:
                    img = cv2.imread(file_path)
                    if img is not None:  # Ensure the image is loaded
                        img_resized = cv2.resize(img, image_size)
                        img_normalized = img_resized / 255.0  # Normalize pixel values
                        images.append(img_normalized)
                        labels.append(class_name)
                except Exception as e:
                    print(f"Error processing file {file_path}: {e}")
    return np.array(images), np.array(labels)

# Define your data directory
data_dir = target  # Use 'target' as defined earlier

# Load and process images
X, y = load_and_process_images(data_dir)

# Print dataset size and shape
print(f"Dataset size: {len(X)} images")
print(f"Image shape: {X[0].shape if len(X) > 0 else 'No images loaded'}")
print(f"Labels size: {len(y)}")

Dataset size: 6500 images
Image shape: (128, 128, 3)
Labels size: 6500


In [4]:
# Encode the labels
encoder = LabelEncoder()
y_encoded = encoder.fit_transform(y)  # Convert string labels to integers
y_categorical = to_categorical(y_encoded)  # Convert to one-hot encoding

# Check label encoding
print(f"Classes: {encoder.classes_}")
print(f"Encoded labels: {y_encoded[:10]}")
print(f"One-hot encoded labels shape: {y_categorical.shape}")

# Split the data (70% training, 30% testing)
X_train, X_test, y_train, y_test = train_test_split(X, y_categorical, test_size=0.3, random_state=42)

# Print dataset shapes
print(f"Training data shape: {X_train.shape}, Test data shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}, Test labels shape: {y_test.shape}")

Classes: ['collie' 'dolphin' 'elephant' 'fox' 'giant+panda' 'moose' 'polar+bear'
 'rabbit' 'sheep' 'squirrel']
Encoded labels: [9 9 9 9 9 9 9 9 9 9]
One-hot encoded labels shape: (6500, 10)
Training data shape: (4550, 128, 128, 3), Test data shape: (1950, 128, 128, 3)
Training labels shape: (4550, 10), Test labels shape: (1950, 10)


In [5]:
datagen = ImageDataGenerator(
    rotation_range=15,  # Rotate images by up to 15 degrees
    width_shift_range=0.1,  # Shift images horizontally by up to 10% of width
    height_shift_range=0.1,  # Shift images vertically by up to 10% of height
    shear_range=0.1,  # Shear images by up to 10%
    zoom_range=0.1,  # Zoom images in/out by up to 10%
    horizontal_flip=True,  # Randomly flip images horizontally
    fill_mode='nearest'  # Fill any missing pixels using the nearest pixel
)
datagen.fit(X_train)

# 2. CNN Model Build

In [6]:
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))  # Pooling layer

# Convolutional layer 2
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))  # Pooling layer

# Flatten the feature maps
model.add(Flatten())

# Output layer
model.add(Dense(10, activation='softmax'))  # 10 classes

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                      │ (None, 126, 126, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d (MaxPooling2D)         │ (None, 63, 63, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 61, 61, 64)          │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_1 (MaxPooling2D)       │ (None, 30, 30, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 57600)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 10)                  │         576,010 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 595,402 (2.27 MB)

 Trainable params: 595,402 (2.27 MB)

 Non-trainable params: 0 (0.00 B)

# 3. CNN Model Test

In [7]:
history = model.fit(
    X_train, y_train,
    validation_split=0.2,  # Use 20% of the training data for validation
    epochs=10,  # Train for 5 epochs
    batch_size=32,  # Use batches of 32 images
    verbose=1
)

test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Epoch 1/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 7s 39ms/step - accuracy: 0.2954 - loss: 1.9905 - val_accuracy: 0.5242 - val_loss: 1.4751
Epoch 2/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5907 - loss: 1.2389 - val_accuracy: 0.5473 - val_loss: 1.3294
Epoch 3/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.7492 - loss: 0.7807 - val_accuracy: 0.5505 - val_loss: 1.4027
Epoch 4/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.8659 - loss: 0.4437 - val_accuracy: 0.5615 - val_loss: 1.4942
Epoch 5/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9462 - loss: 0.2165 - val_accuracy: 0.5571 - val_loss: 1.7108
Epoch 6/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9789 - loss: 0.1021 - val_accuracy: 0.5451 - val_loss: 1.9407
Epoch 7/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9952 - loss: 0.0449 - val_accuracy: 0.5462 - val_loss: 2.2140
Epoch 8/10
114/114 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.9969 - loss: 0.0171 - val_accu

# 4. Manipulation of Images with Different Lights and Testing with Manipulated Test Set

In [8]:
# Simple image manipulation (e.g., increasing contrast)
def manipulate_images(images):
    return np.array([cv2.convertScaleAbs(img, alpha=2.0, beta=0) for img in images])  # Contrast adjustment

# Apply manipulation
X_test_manipulated = manipulate_images(X_test)

# Evaluate the model on manipulated images
manipulated_loss, manipulated_accuracy = model.evaluate(X_test_manipulated, y_test)
print(f"Accuracy on Manipulated Test Images (with contrast): {manipulated_accuracy * 100:.2f}%")

61/61 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.4745 - loss: 7.7571
Accuracy on Manipulated Test Images (with contrast): 47.64%


# 5. Applying the Color Constancy Algorithm to the Manipulated Test Set and Testing with the Color Constancy Test Set

In [9]:
# Gray World algorithm for color constancy
def get_wb_images(images):
    wb_images = []
    
    for img in images:
        # Convert the image to RGB if it's not already
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        
        # Calculate the mean color of the image
        mean_r = np.mean(img_rgb[:,:,0])
        mean_g = np.mean(img_rgb[:,:,1])
        mean_b = np.mean(img_rgb[:,:,2])
        
        # Calculate the average color
        avg = (mean_r + mean_g + mean_b) / 3
        
        # Adjust each channel to balance the colors
        img_rgb[:,:,0] = img_rgb[:,:,0] * (avg / mean_r)
        img_rgb[:,:,1] = img_rgb[:,:,1] * (avg / mean_g)
        img_rgb[:,:,2] = img_rgb[:,:,2] * (avg / mean_b)
        
        # Clip the values to keep them in the 0-255 range
        img_rgb = np.clip(img_rgb, 0, 255)
        
        # Convert back to BGR (if needed) and store
        wb_images.append(cv2.cvtColor(img_rgb.astype(np.uint8), cv2.COLOR_RGB2BGR))
    
    return np.array(wb_images)

# Apply the Gray World algorithm to the manipulated test images
X_test_wb = get_wb_images(X_test_manipulated)

<ipython-input-9-c84537b1bf17>:18: RuntimeWarning: divide by zero encountered in scalar divide
  img_rgb[:,:,0] = img_rgb[:,:,0] * (avg / mean_r)
<ipython-input-9-c84537b1bf17>:18: RuntimeWarning: invalid value encountered in multiply
  img_rgb[:,:,0] = img_rgb[:,:,0] * (avg / mean_r)
<ipython-input-9-c84537b1bf17>:18: RuntimeWarning: invalid value encountered in cast
  img_rgb[:,:,0] = img_rgb[:,:,0] * (avg / mean_r)


In [10]:
# Evaluate the model on the color-corrected test set
wb_loss, wb_accuracy = model.evaluate(X_test_wb, y_test)

# Print the accuracy of the model on the color-constant corrected test set
print(f"Accuracy on Color-Corrected Test Images: {wb_accuracy * 100:.2f}%")

61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0769 - loss: 46.7185
Accuracy on Color-Corrected Test Images: 7.59%


# 6. Comparing and Reporting the Success of Different Test Sets

In [11]:
# Evaluate the model on the original test set
original_loss, original_accuracy = model.evaluate(X_test, y_test)
print(f"Accuracy on Original Test Set: {original_accuracy * 100:.2f}%")

# Evaluate the model on the manipulated test set
manipulated_loss, manipulated_accuracy = model.evaluate(X_test_manipulated, y_test)
print(f"Accuracy on Manipulated Test Set: {manipulated_accuracy * 100:.2f}%")

# Evaluate the model on the color-corrected (Gray World) test set
wb_loss, wb_accuracy = model.evaluate(X_test_wb, y_test)
print(f"Accuracy on Color-Corrected Test Set: {wb_accuracy * 100:.2f}%")


61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5531 - loss: 2.6446
Accuracy on Original Test Set: 55.74%
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4745 - loss: 7.7571
Accuracy on Manipulated Test Set: 47.64%
61/61 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0769 - loss: 46.7185
Accuracy on Color-Corrected Test Set: 7.59%


- Accuracy on Original Test Set: 52.21%

- Accuracy on Manipulated Test Set: 46.31%

- Accuracy on Color-Corrected Test Set: 8.15%

The model performs well on the original test set, but its accuracy drops significantly when exposed to manipulated or color-corrected images.  
This suggests that the model needs further improvement to handle real-world lighting variations.  
Possible solutions include data augmentation, transfer learning, and experimenting with advanced color constancy algorithms.